In [1]:
import numpy as np
import os, sys, cv2, time
import matplotlib.pyplot as plt
import mxnet as mx
from mxnet import gluon, init, autograd, nd, image
from mxnet.gluon import data as gdata, utils as gutils, nn, loss as gloss
from shutil import copyfile
from mxnet.gluon.data.vision import datasets, transforms
from sklearn.metrics import confusion_matrix
from ipywidgets import interact, IntSlider

C:\Users\lab\AppData\Local\Continuum\anaconda3\envs\mxnet-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
# target_folder = 'E:/Guan-Ming/Deep_ML/all_data_resize/'
# output_folder = 'E:/Guan-Ming/Deep_ML/all_data_final_transposed/'
# test_data_folder = 'E:/Guan-Ming/Deep_ML/test_data_resize/'
# extra_validation_target_folder = 'E:/Guan-Ming/Deep_ML/extra_validation_data_resize/'

output_folder = 'Train_Data/'
test_data_folder = 'Test_Data/'
extra_validation_target_folder = 'Extra_Validation_Data/'

### Read in Training Data

In [12]:
files = os.listdir(output_folder)
total_data = len(files)
data_x = nd.zeros((total_data, 256, 256, 3)).astype(np.uint8)
data_y = nd.zeros(total_data)
total_num = 0

for index, f in enumerate(files):
    data_x[index] = image.imread(output_folder+f)
    label = f.split('_',1)[0]
    if label == '0':
        data_y[index] = 0
        total_num += 1
    elif label == '1':
        data_y[index] = 1
        total_num += 1
    elif label == '2':
        data_y[index] = 2
        total_num += 1
    elif label == '3':
        data_y[index] = 3
        total_num += 1
    elif label == '4':
        data_y[index] = 4
        total_num += 1
    elif label == '5':
        data_y[index] = 5
        total_num += 1
    else:
        raise RuntimeError('Cannot label training data!')

if total_num == total_data:
    print("Sum check pass!")

Sum check pass!


### Read in Testing Data

In [13]:
files = os.listdir(test_data_folder)
total_test_data = len(files)
test_data_x = nd.zeros((total_test_data, 256, 256, 3)).astype(np.uint8)
test_data_y = nd.zeros(total_test_data)
total_num = 0

for index in range(total_test_data):
    f = files[index]
    test_data_x[index] = image.imread(test_data_folder+f)
    label = f.split('_', 1)[0]
    if label == '0':
        test_data_y[index] = 0
        total_num += 1
    elif label == '1':
        test_data_y[index] = 1
        total_num += 1
    elif label == '2':
        test_data_y[index] = 2
        total_num += 1
    elif label == '3':
        test_data_y[index] = 3
        total_num += 1
    elif label == '4':
        test_data_y[index] = 4
        total_num += 1
    elif label == '5':
        test_data_y[index] = 5
        total_num += 1
    else:
        raise RuntimeError('Cannot label testation data!')
        
if total_num == total_test_data:
    print("Sum check pass!")

Sum check pass!


### Read in Extra Validation Data

In [14]:
files = os.listdir(extra_validation_target_folder)
total_extra_validation_data = len(files)
extra_validation_data_x = nd.zeros((total_extra_validation_data, 256, 256, 3)).astype(np.uint8)
extra_validation_data_y = nd.zeros(total_extra_validation_data)
total_num = 0

for index, f in enumerate(files):
    extra_validation_data_x[index] = image.imread(extra_validation_target_folder+f)

    label = f.split('_',1)[0]
    if label =='0':
        extra_validation_data_y[index] = 0
        total_num += 1
    elif label =='1':
        extra_validation_data_y[index] = 1
        total_num += 1
    elif label =='2':
        extra_validation_data_y[index] = 2
        total_num += 1
    elif label =='3':
        extra_validation_data_y[index] = 3
        total_num += 1
    elif label =='4':
        extra_validation_data_y[index] = 4
        total_num += 1
    elif label =='5':
        extra_validation_data_y[index] = 5
        total_num += 1
    else:
        raise RuntimeError('Cannot label training data!')
#    break

if total_num == total_extra_validation_data:
    print("Sum check pass!")

Sum check pass!


### Train-Valid Spilt

In [15]:
data_index = np.arange(total_data)
train_size = 0.65
np.random.seed = 61387454
np.random.shuffle(data_index)
train_index = data_index[:np.int32(total_data*train_size)]
valid_index = data_index[np.int32(total_data*train_size):]
train_data_x = data_x[train_index]
train_data_y = data_y[train_index]
valid_data_x = data_x[valid_index]
valid_data_y = data_y[valid_index]

In [16]:
train_mean = [0.592143940485608, 0.5347022953513785, 0.4981124834847486]
train_std = [0.2880660398985942, 0.2820819880842514, 0.282954693408029]
extra_validation_mean = [0.48470740652217936, 0.4379602084974611, 0.37406493007969804]
extra_validation_std = [0.2525134266173694, 0.23321502528915997, 0.2119784063910089]

### Create Data Iterator

In [17]:
batch_size = 64
train_dataset = gdata.ArrayDataset(train_data_x, train_data_y)
valid_dataset = gdata.ArrayDataset(valid_data_x, valid_data_y)
test_dataset = gdata.ArrayDataset(test_data_x, test_data_y)
extra_validation_dataset = gdata.ArrayDataset(extra_validation_data_x, extra_validation_data_y)
    
transformer = []
transformer += [gdata.vision.transforms.ToTensor()] # transer the train data from shape (sample, H, W, channel) to (sample, channel, H, W) and rescale to between 0 and 1 
transformer += [gdata.vision.transforms.Normalize(train_mean, train_std)]
transformer = gdata.vision.transforms.Compose(transformer)
num_workers = 0 if sys.platform.startswith('win32') else 4

train_iter = gdata.DataLoader(train_dataset.transform_first(transformer), batch_size = batch_size, shuffle=True, num_workers=num_workers)
valid_iter = gdata.DataLoader(valid_dataset.transform_first(transformer), batch_size = batch_size, shuffle=False, num_workers=num_workers)
test_iter = gdata.DataLoader(test_dataset.transform_first(transformer), batch_size = batch_size, shuffle=False, num_workers=num_workers)

extra_transformer = []
extra_transformer += [gdata.vision.transforms.ToTensor()] # transer the train data from shape (sample, H, W, channel) to (sample, channel, H, W) and rescale to between 0 and 1 
extra_transformer += [gdata.vision.transforms.Normalize(extra_validation_mean, extra_validation_std)]
extra_transformer = gdata.vision.transforms.Compose(extra_transformer)
extra_validation_iter = gdata.DataLoader(extra_validation_dataset.transform_first(extra_transformer), batch_size = batch_size, shuffle=False, num_workers=num_workers)

### Define Functions for Training

In [18]:
def try_gpu():
    """If GPU is available, return mx.gpu(0); else return mx.cpu()."""
    try:
        ctx = mx.gpu()
        _ = nd.array([0], ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx
def evaluate_accuracy(data_iter, net, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n = nd.array([0]), 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            y = y.astype('float32')
            acc_sum += (net(X).argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n

def _get_batch(batch, ctx):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

def train(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs):
    """Train and evaluate a model with CPU or GPU."""
    print('training on', ctx)
    loss = gloss.SoftmaxCrossEntropyLoss()
    average_time = 0.0
    
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)            
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        epoch_time = time.time() - start
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc, epoch_time))
        average_time += epoch_time
    print('Average time per epoch is %.4f s.'%(average_time/num_epochs))

In [19]:
ctx = try_gpu()

### DENSENET

In [20]:
class DenseBlock(nn.Block): 
    def __init__(self, num_convs, num_channels, **kwargs):
        super(DenseBlock, self).__init__(**kwargs) 
        self.net = nn.Sequential() 
        for _ in range(num_convs): 
            self.net.add(conv_block(num_channels))
            
    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            # Concatenate the input and output of each block on the channel 
            # dimension
            X = nd.concat(X, Y, dim=1) 

        return X
    
def transition_block(num_channels):
    blk = nn.Sequential() 
    blk.add(nn.BatchNorm(), 
            nn.Activation('relu'),
            nn.Conv2D(num_channels, kernel_size=1), 
            nn.AvgPool2D(pool_size=2, strides=2))
    return blk

def conv_block(num_channels): 
    blk = nn.Sequential() 
    blk.add(nn.BatchNorm(), 
            nn.Activation('relu'),
            nn.Conv2D(num_channels, kernel_size=3, padding=1))
    return blk

In [21]:
net_DENSENET = nn.Sequential()
net_DENSENET.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3), 
        nn.BatchNorm(),
        nn.Activation('relu'), 
        nn.MaxPool2D(pool_size=3, strides=2, padding=1))

num_channels, growth_rate = 32, 20
num_convs_in_dense_blocks = [4, 8, 16, 32]

for i, num_convs in enumerate(num_convs_in_dense_blocks):
    net_DENSENET.add(DenseBlock(num_convs, growth_rate))
    net_DENSENET.add(nn.Dropout(0.15))
#     # This is the number of output channels in the previous dense block 
    num_channels += num_convs * growth_rate
    # A transition layer that haves the number of channels is added between
    # the dense blocks 
    if i != len(num_convs_in_dense_blocks) - 1: 
        num_channels //= 2
        net_DENSENET.add(transition_block(num_channels))
                    
net_DENSENET.add(nn.BatchNorm(), 
        nn.Activation('relu'), 
        nn.GlobalAvgPool2D(),
        nn.Dense(6))

### Initialize or Load Weights

In [22]:
net_DENSENET.load_parameters('Weight_Train_Test_Mixed/DENSENET_DROPOUT_train_data_transposed_2.params', ctx= ctx)
#net_DENSENET.load_parameters('C:/Users/lab/Jupyter_Notebook/Guan-Ming/Midterm_Project/NN_params/train_test_mixed/DENSENET_DROPOUT_train_data_transposed_2.params', ctx = ctx)
#net_DENSENET.initialize(ctx=ctx, init=mx.init.Xavier(rnd_type ='gaussian', factor_type='in', magnitude=2.0))

In [23]:
for X,_ in train_iter:
    net_DENSENET.summary(X.as_in_context(ctx))
    break

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                           (64, 3, 256, 256)               0
            Conv2D-1                          (64, 64, 128, 128)            9472
         BatchNorm-2                          (64, 64, 128, 128)             256
        Activation-3                          (64, 64, 128, 128)               0
         MaxPool2D-4                            (64, 64, 64, 64)               0
         BatchNorm-5                            (64, 64, 64, 64)             256
        Activation-6                            (64, 64, 64, 64)               0
            Conv2D-7                            (64, 20, 64, 64)           11540
         BatchNorm-8                            (64, 84, 64, 64)             336
        Activation-9                            (64, 84, 64, 64)               0
           Conv2D-10        

### Training

In [ ]:
adam_optimizer = mx.optimizer.Adam(learning_rate=1e-4, beta1=0.9, beta2=0.9)
trainer = gluon.Trainer(net_DENSENET.collect_params(), optimizer=adam_optimizer)
train(net_DENSENET, train_iter, valid_iter, batch_size, trainer, ctx, 10)

### Print Accuracy for Testing Data Set

In [24]:
print('Accuracy of model is %.4f %% .'%(100*evaluate_accuracy(test_iter, net_DENSENET, ctx)))

Accuracy of model is 75.0000 % .


### Print Accuracy for Extra Validation Data Set 

In [25]:
print('Accuracy of model is %.4f %% .'%(100*evaluate_accuracy(extra_validation_iter, net_DENSENET, ctx)))

Accuracy of model is 33.9286 % .


### Data Saving and Network Structure

In [ ]:
#net_DENSENET.save_parameters('C:/Users/lab/Jupyter_Notebook/Guan-Ming/Midterm_Project/NN_params/train_test_mixed/DENSENET_DROPOUT_train_data_transposed_2.params')
#===============================================================
# net_DENSENET = nn.Sequential()
# net_DENSENET.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3), 
#         nn.BatchNorm(),
#         nn.Activation('relu'), 
#         nn.MaxPool2D(pool_size=3, strides=2, padding=1))

# num_channels, growth_rate = 32, 16
# num_convs_in_dense_blocks = [4, 8, 16, 32, 64]

# for i, num_convs in enumerate(num_convs_in_dense_blocks):
#     net_DENSENET.add(DenseBlock(num_convs, growth_rate))
#     net_DENSENET.add(nn.Dropout(0.15))
# #     # This is the number of output channels in the previous dense block 
#     num_channels += num_convs * growth_rate
#     # A transition layer that haves the number of channels is added between
#     # the dense blocks 
#     if i != len(num_convs_in_dense_blocks) - 1: 
#         num_channels //= 2
#         net_DENSENET.add(transition_block(num_channels))
                    
# net_DENSENET.add(nn.BatchNorm(), 
#         nn.Activation('relu'), 
#         nn.GlobalAvgPool2D(),
#         nn.Dense(6))
#===============================================================

In [27]:
y_pred_test = np.zeros(total_test_data)
count = 0
for X, _ in test_iter:
    y_pred_test[count:min(count+batch_size, total_test_data)] = net_DENSENET(X.as_in_context(ctx)).asnumpy().argmax(axis=1)
    count += batch_size
wrong_index_test = np.argwhere(y_pred_test!=test_data_y.asnumpy())

In [28]:
result = confusion_matrix(test_data_y.asnumpy(), y_pred_test)
print(result)
print('\n')
for i in range(6):
    print("Accuracy of %d is %.2f %%." %(i, 100*result[i][i]/result[i].sum()))

[[46  4  0  0  0  0]
 [ 5 44  0  0  0  1]
 [ 1  9 34  6  0  0]
 [ 1  7  0 38  3  1]
 [ 1  3  3  4 26 13]
 [ 0  3  1  6  3 37]]


Accuracy of 0 is 92.00 %.
Accuracy of 1 is 88.00 %.
Accuracy of 2 is 68.00 %.
Accuracy of 3 is 76.00 %.
Accuracy of 4 is 52.00 %.
Accuracy of 5 is 74.00 %.


In [29]:
def PLOT_WRONG_PREDICTION(index_selected):
    print(index_selected)
    plt.figure(figsize=(3,3), dpi=120)
    plt.imshow(test_data_x[wrong_index_test[index_selected]][0].asnumpy().astype(np.uint8))
    plt.axis('off')
    plt.title("NN Recognizes as %d .\nReal Answer is %d ." %(y_pred_test[wrong_index_test[index_selected]], test_data_y[wrong_index_test[index_selected]].asscalar()) )
    
print('Total %d wrong predcition. Slide to see the corresponding image.' %len(wrong_index_test))
interact(PLOT_WRONG_PREDICTION, index_selected=IntSlider(value=0, min=0, max=len(wrong_index_test)-1));

Total 75 wrong predcition. Slide to see the corresponding image.


interactive(children=(IntSlider(value=0, description='index_selected', max=74), Output()), _dom_classes=('widg…

### Confusion Matrix of Extra Validation Data Set

In [31]:
y_pred_extra_validation = np.zeros(total_extra_validation_data)
count = 0
for X, _ in extra_validation_iter:
    y_pred_extra_validation[count:count+batch_size] = net_DENSENET(X.as_in_context(ctx))[:total_extra_validation_data].asnumpy().argmax(axis=1)
    count += batch_size
wrong_index_extra_validation = np.argwhere(y_pred_extra_validation!=extra_validation_data_y.asnumpy())

In [32]:
result = confusion_matrix(extra_validation_data_y.asnumpy(), y_pred_extra_validation)
print(result)
print('\n')
for i in range(6):
    print("Accuracy of %d is %.2f %%." %(i, 100*result[i][i]/result[i].sum()))

[[2 0 0 0 0 0]
 [1 4 0 1 0 0]
 [0 0 3 2 0 1]
 [1 2 6 9 2 7]
 [0 0 2 2 0 1]
 [0 1 2 6 0 1]]


Accuracy of 0 is 100.00 %.
Accuracy of 1 is 66.67 %.
Accuracy of 2 is 50.00 %.
Accuracy of 3 is 33.33 %.
Accuracy of 4 is 0.00 %.
Accuracy of 5 is 10.00 %.


In [33]:
def PLOT_WRONG_PREDICTION(index_selected):
    print(index_selected)
    plt.figure(figsize=(3,3), dpi=120)
    plt.imshow(extra_validation_data_x[wrong_index_extra_validation[index_selected]][0].asnumpy().astype(np.uint8))
    plt.axis('off')
    plt.title("NN Recognizes as %d .\nReal Answer is %d ." %(y_pred_extra_validation[wrong_index_extra_validation[index_selected]], extra_validation_data_y[wrong_index_extra_validation[index_selected]].asscalar()) )
    
print('Total %d wrong predcition. Slide to see the corresponding image.' %len(wrong_index_extra_validation))
interact(PLOT_WRONG_PREDICTION, index_selected=IntSlider(value=0, min=0, max=len(wrong_index_extra_validation)-1));

Total 37 wrong predcition. Slide to see the corresponding image.


interactive(children=(IntSlider(value=0, description='index_selected', max=36), Output()), _dom_classes=('widg…